# Data Mining - Handin 2 - Graph mining

Welcome to the handin on clustering algorithms and outlier detection. 
This handin corresponds to the topics in Week 10--15 in the course.

The handin IS 
* mandatory
* done individually
* worth 10% of the grade

For the handin, you will prepare a report in PDF format, by exporting the Jupyter notebook. 
Please submit
1. The jupyter notebook file with your answers
2. The PDF obtained by exporting the jupyter notebook

Submit both files on Blackboard no later than **April 20th kl. 13.59**.

In [ ]:
### DO NOT TOUCH THESE IMPORTS
import sys
sys.path.append('..')

import random
import scipy.io as sio
import time

import networkx as nx
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from utilities.load_data import load_mnist

## 1. Theoretical questions

We will start out with theory questions. These questions are theoretical; as such, they **can and should be answered without implementation**. In particular, you are allowed to verify your answer experimentally, but the motivation you provide should be only in text and theoretically grounded. 

### Spectral theory and graph partitioning

Let $L = \Delta - A$ the Laplacian matrix of a graph $G$ and be $\lambda_1 \geq \lambda_2 ... \geq \lambda_n$ be the eigenvalues of $L$ with eigenvectors $\textbf{u}_1, ..., \textbf{u}_n$. (Notice this is differenty defined from what was used in the weekly exercises)

1. Consider a dataset composed of four points. Assume we compute a similarity matrix uses a threshold function on Euclidean (norm-2) distance. The metric outputs 1 if the points are close enough according to a threshold and zero otherwise. Consider two cases: 
    1. When there are two pairs of points close to each other (see image below, left)
    2. When there is only one point isolated from the others (see image below, right)
    <div>
    <img src="images/spectral_1.png" width="300"><img src="images/spectral_2.png" width="300">
    </div>
    2. Build the Laplacian in each case and discuss the eigenvalues and eigenvectors.
2. Consider the graph below that has no edge among nodes (see picture below). In such a graph, compute the modularity as a function of the number of nodes, of a clustering $S$ which assigns each cluster a single node. 
    
    ![image.png](images/modularity.png)
3. Given the modularity definition $\displaystyle Q(G,\mathcal{C})=\frac{1}{2 m} \sum_{c \in \mathcal{C}} \sum_{i \in c} \sum_{j \in c}\left(a_{i j}-\frac{d_{i} d_{j}}{2 m}\right)$, what is the partition with _minimum_ modularity? Motivate your answer. 
4. Prove that the modularity of a partition having all nodes in one cluster is $\frac{1}{4}$ irrespective of the number of nodes of the graph $G$.
5.  Let $G$ be a graph with largest Laplacian eigenvalue $\lambda_1$ and maximum degree $\Delta > 0$. Show that $\lambda_1 \geq \Delta + 1$.
 

#### Your Answers

### Random walks and PageRank
 
Recall that the PageRank is defined as $\mathbf{r} = \alpha \mathbf{Mr} + (1-\alpha)\mathbf{p}$, where $\mathbf{r}$ is the PageRank vector, $\alpha$ is the restart probability, $\mathbf{M} = A^\top\Delta^{-1}$, and $\mathbf{p}$ is the restart (or personalization) vector. 


1. What is the PageRank of a complete bipartite graph with 3 nodes on one side and 4 on the other side and $\alpha=1$?  
2. You are given a complete subgraph (i.e., a clique) with a dangling node (number 1, in the image below)
    1. Compute, __and show the computation steps__, the PageRank (i.e. $\alpha=1$) for the graph below when the number of nodes $n=4$ and the clique has 3 nodes. **Note**: In this question you should compute PageRank by hand solving the system of equations or via Power Iteration
    2. Show what happens to the PageRank when you increase by 1 node the size of the clique in the graph in point 1. What is the PageRank of the nodes __not__ connected with the isolated node? What is the PageRank of the isolated node? **Note**: No computation needed, only reasoning 
    3. Assume now the graph has $n$ nodes (where $n$ is a variable) and a clique with $n-1$ nodes. What is the PageRank of  the $n-2$ nodes in the clique that are not connected to the isolated node? Provide sufficient motivation for your argument or, if you can, a formula which depends on $n$.
    4. Assume you have embedded the graph with a __Linear Embedding__ using unormalized Laplacian matrix of the graph as the similarity matrix. How do you expect the embeddings to be if the embedding dimension is $d = 1$? Motivate your answer with a mathematical reasoning.
    
    ![image.png](images/pagerank.png)

#### Your Answers

## 2. Spectral clustering
We will now proceed with community detection algorithms. 
For this part of the hand in, we will try to cluster a subset of the MNIST data set.

In [ ]:
X_train, y_train, *_ = load_mnist()
np.random.seed(1)  # DO NOT CHANGE

n, h, w = X_train.shape
sel = np.random.permutation(n)
X = X_train[sel]
y = y_train[sel]

fig, ax = plt.subplots(2,6, figsize=(10, 3))
for i in range(2): 
    for j in range(6):
        ax[i, j].imshow(X[i*4 + j])
        ax[i, j].set_title("Label: %d" % y[i*4 + j])
plt.tight_layout()

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Load 200 random samples from the MNIST training set.
X_ = X[:200].reshape(-1, 784)
X_ = (X_ - X_.mean(1, keepdims=True)) / X_.std(1, keepdims=True)
y_ = y[:200]

def plot_neighborhood_graph(G, pos, ax):
    ax.axis('off')
    ax.set_aspect('equal')
    
    nx.draw_networkx_edges(G,pos=pos,ax=ax, alpha=0.3)
    nx.draw_networkx_nodes(G,pos=pos,ax=ax, node_color=y_, node_size=50, cmap=plt.get_cmap('tab10'))
    
    ax.set_xlim(-1.1,1.1)
    ax.set_ylim(-1.1,1.1)
    
def plot_img_neighborhood_graph(G, X, pos, ax):
    
    ax.axis('off')
    ax.set_aspect('equal')
    
    trans=ax.transData.transform
    trans2=fig.transFigure.inverted().transform
    
    nx.draw_networkx_edges(G, pos=pos, ax=ax, alpha=0.3)
    
    ax.set_xlim(-1.1,1.1)
    ax.set_ylim(-1.1,1.1)
    
    # Add images to graph
    piesize=0.02 # this is the image size
    p2=piesize/2.0
    for n in G.nodes:
        xx,yy=trans(pos[n]) # figure coordinates
        xa,ya=trans2((xx,yy)) * np.array([1.001, 0.93]) + np.array([0., 0.04]) # axes coordinates
        a = plt.axes([xa-p2,ya-p2, piesize, piesize])
        # a = plt.axes([xa-p2,ya-p2, piesize, piesize])
        a.set_aspect('equal')
        a.imshow(X_[n].reshape(28, 28))
        a.axis('off')
        

1. Take the dataset and construct the $\varepsilon$-neighborhood graph, using Eucledian (L2) distance. Try with different epsilons and plot the graphs. What do you observe as epsilon increases? You can use the code below to plot the graph if you wish. Fix the $\varepsilon = 20$ in the rest of the exercise.

In [ ]:
# You are allowed to use sklearn if you wish. 
# Be sure that your constructed graphs does not contain loop edges (edges from i to i for some node i)
from sklearn.neighbors import NearestNeighbors 


def nn_graph(X, eps=20):
    G = None
    ### YOUR CODE
    
    ### YOUR CODE
    return G


In [ ]:
# TODO: Experiment with eps-neighborhood graph for different values of eps. You can use code below for plotting

# Use the reduced data-set X_. Otherwise computation and plotting will take much too long.
G = nn_graph(X_)
pos=nx.spring_layout(G)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(121)
plot_neighborhood_graph(G, pos, ax)

ax=plt.subplot(122)
plot_img_neighborhood_graph(G, X, pos, ax)

#### Your answer

2. Assign to each edge in the $\varepsilon$-neighborhood graph a weight
    $$W_{i j}=e^{-\frac{\left\|\mathbf{x}_{i}-\mathbf{x}_{j}\right\|^{2}}{t}}$$
   where $t\geq 0$ is a parameter. What happens when $t$ is very small, close to $0$, i.e., $t \rightarrow 0$? What happens when $t$ is very large? 

In [ ]:
def weighted_nn_graph(X, eps=20, t=0.01):
    G = None
    ### YOUR CODE
    
    ### YOUR CODE
    return G

#### Your answer

3. Compare the eigenvectors of the weighted adjacency matrix, the weighted normalized laplacian, and the weighted random walk laplacian. Plot the eigenvectors of the different matrices. How do they relate? How do the matrix values vary with $t$?

In [ ]:
### YOUR CODE HERE

#### Your answer

4. Run spectral clustering with the weighted random walk and the weighted normalized laplacian with $t = 0.01$. You should implement spectral clustering yourself. You are allowed to use your implementation from the weekly exercises.


In [ ]:
### YOUR CODE HERE

5. Record the performance with Normalized Mutual Information (NMI) you implemented in the first Handin. What is the value of $t \in \{0.001,0.01,0.1,1,10,100,1000\}$ and a number of communities $k \in \{2,3,5,10\}$ that maximizes the NMI. 

In [ ]:
### YOUR CODE HERE

6. There seems to be a relationships between embeddings and spectral clustering, can you guess that? _Hint_: Think to the eigenvectors of the graph's Laplacians.

#### Your answer

## 3. Link analysis
In this exercise, we will work with PageRank, Random Walks and their relationships with graph properties. 
We will use the most generic definition

$$\mathbf{r} = \alpha \mathbf{Mr} + (1-\alpha)\mathbf{p}$$

with $\mathbf{r}$ the PageRank vector, $\mathbf{M}$ the weighted transition matrix, and $\mathbf{p}$ the personalization vector. Additionally, let $n = |V|$, where $V$ is the nodes in the graph above (should be 200 nodes). 


1. Define and compute the transition (PageRank) matrix for the weighted graph in the previous exercise.

In [ ]:
### YOUR CODE HERE

2. Use the weighted graph constructed in the previous exercise with the values of $\varepsilon=20$ and $k$ according to best NMI, run PageRank with $\alpha = 0.85$, and vary $t \in \{0,0.001,0.01,0.1,1,10,100,1000\}$ (test all values in the range). What do you notice as $t$ varies? With different values of $t$, what is the node with the highest PageRank? What are the 5 nodes with the lowest PageRank? How do they change with t? 

In [ ]:
### YOUR CODE HERE

#### Your Answer

3. In the same graph, run PageRank with $t=0.01$ and $\alpha \in \{0, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1\}$. Do you notice some relationship between $t$ and $\alpha$ and PageRank? How do you explain that? 

In [ ]:
### YOUR CODE HERE

#### Your Answer

We will now study the effect of spam in the network and construct a link farm. For simplicity treat all weights in the graph as binary. That is, $w_{ij} = 1$ if $j$ is in the $\varepsilon$-neighborhood of $i$, i.e., $j \in N_\varepsilon(i)$, and $w_{ij}=0$ otherwise. 

1. Based on the analysis in the slides, construct a spam farm $s$ on the graph above with $T$ fake nodes. Assume that $s$ manages to get links from all the node that refers to the digit 1. With $\alpha=0.85$, how many pages $T$ do you need to add in order to get $s$ being assigned the highest PageRank? Provide sufficient theoretical justification. To help your analysis you can initially provide a piece of code that finds $T$ such that the PageRank $\mathbf{r}_s$ is maximizied. 

In [ ]:
### YOUR CODE HERE

#### Your Answer

2. In the above scenario, assume that $T = \frac{1}{5}$ of the nodes in the original graph. What value of $\alpha$ will maximize the PageRank $\mathbf{r}_s$ of the link farm $s$? Provide sufficient justification for your choice. 

In [ ]:
### YOUR CODE HERE

#### Your Answer

3. Now we fix both $\alpha = 0.85$ and $T = \frac{1}{5}n$. Implement the method for spam mass estimation and check whether it is able to detect the node $s$, if the trusted set of nodes is a random sample $10\%$ of the nodes in the original graph. If not, what could be a viable solution? Which nodes would you rather choose as trusted? 

In [ ]:
### YOUR CODE HERE

#### Your Answer

## 4. Graph embeddings

In this final exercise, we will try a different approach for clustering the data from above. Assume the graph is **not** weighted and use the graph with $\varepsilon = 20$.
The strategy is going to be the following:

1. Use VERSE [[1]](https://arxiv.org/pdf/1803.04742.pdf) to produce embeddings of the nodes in the graph.
2. Use K-Means to cluster the embeddings. Measure and report NMI for the clustering. 
3. Now assume the graph is weighted with weights in exercise 2. Tune the parameter $t$ to obtain the best NMI. What do you notice in terms of weights? In particular, is $t$ large or small and what does it imply for the structure? 

Your task is to i) fill in the methods below to implement the sampling version of VERSE. _Hint:_ it might be a help to look in the original article.
and ii) run K-means on the embeddings to evaluate the performance compared to Spectral clustering using the NMI as measure.

[[1](https://arxiv.org/pdf/1803.04742.pdf)] Tsitsulin, A., Mottin, D., Karras, P. and Müller, E., 2018, April. Verse: Versatile graph embeddings from similarity measures. In Proceedings of the 2018 World Wide Web Conference (pp. 539-548).

In [ ]:
def sigmoid(x):
    ''' Return the sigmoid function of x 
        x: the input vector
    '''
    ### YOUR CODE HERE
    ### YOUR CODE HERE
    return x

def pagerank_matrix(G, alpha = 0.85) :     
    ''' Return the Personalized PageRank matrix of a graph

        Args:
            G: the input graph
            alpha: the dumping factor of  PageRank

        :return The nxn PageRank matrix P
    '''
    ### YOUR CODE HERE
    ### YOUR CODE HERE
    return P
    

def update(u, v, Z, C, step_size) :
    '''Update the matrix Z using row-wise gradients of the loss function

       Args:
            u : the first node
            v : the second node
            Z : the embedding matrix
            C : the classification variable used in Noise Contrastive estimation indicating whether the sample is positive or negative
            step_size: step size for gradient descent


       :return nothing, just update rows Z[v,:] and and Z[u,:]
    '''
    ### YOUR CODE HERE
    ### YOUR CODE HERE
    
    
def verse(G, S, d, k = 3, step_size = 0.0025, steps = 10000): 
    ''' Return the sampled version of VERSE

        Args:
            G: the input Graph
            S: the PageRank similarity matrix
            d: dimension of the embedding space
            k: number of negative samples
            step_size: step size for gradient descent
            steps: number of iterations

        :return the embedding matrix nxd
    '''
    n = G.number_of_nodes()
    Z = 1/d*np.random.rand(n,d)

    ### YOUR CODE HERE
    ### YOUR CODE HERE
    
    return Z


In [ ]:
# This code runs the `verse` algorithm above on G and stores the embeddings to 'verse.npy'
P   = pagerank_matrix(G)
emb = verse(G, P, 128, step_size=0.0025, steps=10_000)
np.save('verse.npy', emb)

# TODO
# Run K-means multiple times and choose the best.
# Compare to Spectral clustering on G, according to NMI 